In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('xxx.csv')

In [5]:
df = df[['question', 'result', 'next_word', 'next_two_word']]

In [25]:
df.sample()

,question,result,next_word,next_two_word
9722,不过，在美国国会中诸如钱布利斯等反对停产F-22战机及其部件的议员大有人在，他们认为由多少架...,187,架飞机,组成


In [27]:
next_word = df['next_word'].values.tolist()

In [31]:
set(next_word)

{' ',
 '"',
 '(',
 ')',
 ',',
 '-',
 '.',
 ':',
 '°',
 '”',
 '、',
 '。',
 '一直',
 '万',
 '万人次',
 '万亿',
 '万亿日元',
 '万亿美元',
 '万亿韩元',
 '万份',
 '万余',
 '万余名',
 '万元',
 '万公里',
 '万千克',
 '万千瓦',
 '万千米',
 '万台',
 '万名',
 '万吨',
 '万吨级',
 '万多',
 '万多千米',
 '万多名',
 '万多枚',
 '万多辆',
 '万封',
 '万左右',
 '万平方公里',
 '万平方米',
 '万枚',
 '万枝',
 '万架',
 '万桶',
 '万次',
 '万海里',
 '万磅',
 '万米',
 '万美元',
 '万英尺',
 '万英镑',
 '万行',
 '万辆',
 '万韩元',
 '三发',
 '三款',
 '上',
 '上海',
 '下',
 '不',
 '不仅',
 '不会',
 '专机',
 '世纪',
 '两',
 '两栖',
 '两种',
 '个',
 '个人',
 '个子',
 '中',
 '中央',
 '中航',
 '中队',
 '为',
 '主要',
 '之后',
 '之间',
 '也',
 '事件',
 '于',
 '交互',
 '亮相',
 '人',
 '人为',
 '人均',
 '人次',
 '人生',
 '亿',
 '亿个',
 '亿元',
 '亿公斤',
 '亿台',
 '亿吨公里',
 '亿日元',
 '亿次',
 '亿澳元',
 '亿美金',
 '亿英镑',
 '亿韩元',
 '仅',
 '从',
 '代',
 '以来',
 '件',
 '任',
 '份',
 '份文件',
 '伤',
 '位',
 '体积',
 '余',
 '余个',
 '余名',
 '余吨',
 '余年',
 '余枚',
 '余架',
 '余次',
 '余米',
 '余艘',
 '余项',
 '作为',
 '作战',
 '使',
 '使用',
 '例',
 '侦察机',
 '保护',
 '倍',
 '停产',
 '像素',
 '元',
 '兆',
 '兆比特',
 '兆瓦',
 '兆赫',
 '克',
 '克拉',
 '全球',
 '全面',
 '公分',
 '公开'